### Import Libraries

In [ ]:
# import sys
# !{sys.executable} -m pip install gensim

In [1]:
import re
import requests
import pandas as pd
import numpy as np
import datetime
import dateutil

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances

In [2]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [3]:
# print(datetime.datetime.strptime('07/28/2014 18:54:55.099000', '%m/%d/%Y %H:%M:%S.%f'))
# print(dateutil.parser.parse('2021-03-19T14:02:21.009532-04:00'))
# print(datetime.datetime.now(dateutil.tz.tzoffset(None, -14400)))

### Database Setup

In [4]:
def makeDB(database_filename):
    conn = sqlite3.connect(database_filename)
    c = conn.cursor()
    c.execute('''CREATE TABLE RECOMMENDED_MODEL
             ([ID] INTEGER PRIMARY KEY AUTOINCREMENT, [HEADLINE] TEXT, [NEWS_SOURCE] TEXT, [PUBLISHED] DATE, [DESCRIPTION] TEXT, [AUTHOR] TEXT, [IMAGE_URL] TEXT, [LINK] TEXT, [TRENDING_SCORE] REAL)''')
    conn.commit()
    conn.close()
    
def saveToDatabase(df, database_filename):
    conn = sqlite3.connect(database_filename)
    c = conn.cursor()
    df.to_sql('RECOMMENDED_MODEL', conn, if_exists='append', index = False)
    conn.commit()
    conn.close()

def getHeadlines():
    conn = sqlite3.connect('Articles.db')
    c = conn.cursor() 
    c.execute('''SELECT HEADLINE FROM RECOMMENDED_MODEL''')
    headlines = c.fetchall()
    conn.close()
    return headlines

In [5]:
import os
import sqlite3

DATABASE_FILENAME = 'Articles.db'
if not os.path.isfile(DATABASE_FILENAME):
    makeDB(DATABASE_FILENAME)

### Article Retrieve

In [6]:
database_healines = getHeadlines()
database_healines = [x[0] for x in database_healines]
database_healines

[]

In [7]:
URL = 'http://mertium.com/api/news/trending/articles/last/week/'
r = requests.get(url = URL)
data = r.json()

REPLACE_BY_SPACE_RE = re.compile('/|-|:')

articles = []
repeated_articles = []
db_articles = []
titles_set = set()
headlines, news_source, published, description, author, image_url, link, trending_score  = [], [], [], [], [], [], [], []



while True:
    print(URL)
    for i, article in enumerate(data['results']):
        
        headline = article['title']
        headline = headline.replace('&#39;', '\'')
        headline = REPLACE_BY_SPACE_RE.sub(' ', headline)

        if article['title'] in titles_set:
            repeated_articles.append(article['title'])
        elif (headline in database_healines) or (article['title'] in database_healines):
            db_articles.append(article['title'])
        else:
            time = dateutil.parser.parse(data['results'][i]['timestamp'])
            
            headlines.append(data['results'][i]['title'])
            news_source.append(data['results'][i]['news_source'])
            published.append(data['results'][i]['timestamp'])
            description.append(data['results'][i]['description'])
            if len(data['results'][i]['authors']) == 0:
                author.append('')
            else:
                author.append(data['results'][i]['authors'][0]['name'])
                
            image_url.append(data['results'][i]['image'])
            link.append(data['results'][i]['link'])
            trending_score.append(data['results'][i]['trending_score'])
                
            articles.append(article)
        titles_set.add(article['title'])

        
        
    URL = data['next']
    r = requests.get(url = URL)
    data = r.json()
    
    if data['next'] == None:
        
        for i, article in enumerate(data['results']):
            
            headline = article['title']
            headline = headline.replace('&#39;', '\'')
            headline = REPLACE_BY_SPACE_RE.sub(' ', headline)

            if article['title'] in titles_set:
                repeated_articles.append(article['title'])
            elif (headline in database_healines) or (article['title'] in database_healines):
                db_articles.append(article['title'])
            else:
                time = dateutil.parser.parse(data['results'][i]['timestamp'])

                headlines.append(data['results'][i]['title'])
                news_source.append(data['results'][i]['news_source'])
                published.append(data['results'][i]['timestamp'])
                description.append(data['results'][i]['description'])
                if len(data['results'][i]['authors']) == 0:
                    author.append('')
                else:
                    author.append(data['results'][i]['authors'][0]['name'])

                image_url.append(data['results'][i]['image'])
                link.append(data['results'][i]['link'])
                trending_score.append(data['results'][i]['trending_score'])

                articles.append(article)
            titles_set.add(article['title'])
            
        break
print('# of Articles:          ', len(articles)) 
print('# of Repeated Articles: ', len(repeated_articles))
print('# of DB Articles: ', len(db_articles))

http://mertium.com/api/news/trending/articles/last/week/
http://mertium.com/api/news/trending/articles/last/week/?page=2
http://mertium.com/api/news/trending/articles/last/week/?page=3
http://mertium.com/api/news/trending/articles/last/week/?page=4
http://mertium.com/api/news/trending/articles/last/week/?page=5
http://mertium.com/api/news/trending/articles/last/week/?page=6
http://mertium.com/api/news/trending/articles/last/week/?page=7
http://mertium.com/api/news/trending/articles/last/week/?page=8
http://mertium.com/api/news/trending/articles/last/week/?page=9
http://mertium.com/api/news/trending/articles/last/week/?page=10
http://mertium.com/api/news/trending/articles/last/week/?page=11
http://mertium.com/api/news/trending/articles/last/week/?page=12
http://mertium.com/api/news/trending/articles/last/week/?page=13
http://mertium.com/api/news/trending/articles/last/week/?page=14
http://mertium.com/api/news/trending/articles/last/week/?page=15
http://mertium.com/api/news/trending/arti

In [8]:
REPLACE_BY_SPACE_RE = re.compile('/|-|:')

articles = []
for index, headline in enumerate(headlines):
    article = {}
    headline = headline.replace('&#39;', '\'')
    headline = REPLACE_BY_SPACE_RE.sub(' ', headline)
    article['HEADLINE'] = headline
    article['NEWS_SOURCE'] = news_source[index]
    article['PUBLISHED'] = published[index]
    article['DESCRIPTION'] = description[index]
    article['AUTHOR'] = author[index]
    article['IMAGE_URL'] = image_url[index]
    article['LINK'] = link[index]
    article['TRENDING_SCORE'] = trending_score[index]
    articles.append(article)
articles

[{'HEADLINE': 'Tesla accused of environmental rule violations in U.S. and Germany',
  'NEWS_SOURCE': 'CNBC',
  'PUBLISHED': '2021-04-28T10:31:12.850178-04:00',
  'DESCRIPTION': 'The EPA accused Tesla last week of failing to prove it is in compliance with federal emission standards for hazardous air pollutants.',
  'AUTHOR': 'Lora Kolodny',
  'IMAGE_URL': 'https://mertium.s3.amazonaws.com/media/media/106798586-1605671404417-gettyimages-1228033083-TELSA_FREMONT.jpegv1605670992',
  'LINK': 'https://www.cnbc.com/2021/04/28/tesla-accused-of-environmental-rule-violations-in-us-and-germany.html',
  'TRENDING_SCORE': 419.80428396594},
 {'HEADLINE': 'U.K. Must Create More Projects to Attract Investment  Grimstone',
  'NEWS_SOURCE': 'Bloomberg',
  'PUBLISHED': '2021-04-28T07:19:57.024376-04:00',
  'DESCRIPTION': 'Source: Bloomberg, 21:47',
  'AUTHOR': '',
  'IMAGE_URL': 'https://mertium.s3.amazonaws.com/media/media/bloomberg_default-a4f15fa7ee_2qdBpQe.jpg',
  'LINK': 'https://www.bloomberg.com/n

In [9]:
df = pd.DataFrame.from_records([s for s in articles])
df.head()

,HEADLINE,NEWS_SOURCE,PUBLISHED,DESCRIPTION,AUTHOR,IMAGE_URL,LINK,TRENDING_SCORE
0,Tesla accused of environmental rule violations...,CNBC,2021-04-28T10:31:12.850178-04:00,The EPA accused Tesla last week of failing to ...,Lora Kolodny,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-accused-...,419.804284
1,U.K. Must Create More Projects to Attract Inve...,Bloomberg,2021-04-28T07:19:57.024376-04:00,"Source: Bloomberg, 21:47",,https://mertium.s3.amazonaws.com/media/media/b...,https://www.bloomberg.com/news/audio/2021-04-2...,400.000000
2,"Covid Stricken India Receives Ventilators, Int...",Forbes,2021-04-28T08:22:58.750101-04:00,"Several countries including the U.S., U.K., Ru...",Siladitya Ray,https://mertium.s3.amazonaws.com/media/media/0...,https://www.forbes.com/sites/siladityaray/2021...,402.000000
3,Goldman Sachs thinks the U.K. economy will gro...,MarketWatch,2021-04-28T06:16:04.702386-04:00,Early indications show the country is bouncing...,Rupert Steiner,https://mertium.s3.amazonaws.com/media/media/s...,https://www.marketwatch.com/story/goldman-sach...,421.099433
4,Tesla is now sitting on $2.5 billion of bitcoin,CNBC,2021-04-28T07:19:17.623325-04:00,The company announced earlier this year that i...,Jesse Pound,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-is-now-s...,400.800000


In [10]:
df['PUBLISHED'] = pd.to_datetime(df['PUBLISHED'], infer_datetime_format=True)
df.head()

,HEADLINE,NEWS_SOURCE,PUBLISHED,DESCRIPTION,AUTHOR,IMAGE_URL,LINK,TRENDING_SCORE
0,Tesla accused of environmental rule violations...,CNBC,2021-04-28 10:31:12.850178-04:00,The EPA accused Tesla last week of failing to ...,Lora Kolodny,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-accused-...,419.804284
1,U.K. Must Create More Projects to Attract Inve...,Bloomberg,2021-04-28 07:19:57.024376-04:00,"Source: Bloomberg, 21:47",,https://mertium.s3.amazonaws.com/media/media/b...,https://www.bloomberg.com/news/audio/2021-04-2...,400.000000
2,"Covid Stricken India Receives Ventilators, Int...",Forbes,2021-04-28 08:22:58.750101-04:00,"Several countries including the U.S., U.K., Ru...",Siladitya Ray,https://mertium.s3.amazonaws.com/media/media/0...,https://www.forbes.com/sites/siladityaray/2021...,402.000000
3,Goldman Sachs thinks the U.K. economy will gro...,MarketWatch,2021-04-28 06:16:04.702386-04:00,Early indications show the country is bouncing...,Rupert Steiner,https://mertium.s3.amazonaws.com/media/media/s...,https://www.marketwatch.com/story/goldman-sach...,421.099433
4,Tesla is now sitting on $2.5 billion of bitcoin,CNBC,2021-04-28 07:19:17.623325-04:00,The company announced earlier this year that i...,Jesse Pound,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-is-now-s...,400.800000


In [11]:
len(df)

890

In [12]:
df.dtypes

HEADLINE                                          object
NEWS_SOURCE                                       object
PUBLISHED         datetime64[ns, pytz.FixedOffset(-240)]
DESCRIPTION                                       object
AUTHOR                                            object
IMAGE_URL                                         object
LINK                                              object
TRENDING_SCORE                                   float64
dtype: object

In [13]:
saveToDatabase(df, 'Articles.db')

### Data Preprocessing

In [ ]:
# df.replace('', np.nan, inplace = True)
# df.head()

In [ ]:
# # df.sort_values('headline',inplace=True, ascending=False)
# repeated_articles = df.duplicated('title', keep = "first") # keep=False
# df = df[~repeated_articles]
# df.index = range(df.shape[0])
# df.head()

In [14]:
df.shape

(890, 8)

In [15]:
dfx = df.copy()
dfx.head()

,HEADLINE,NEWS_SOURCE,PUBLISHED,DESCRIPTION,AUTHOR,IMAGE_URL,LINK,TRENDING_SCORE
0,Tesla accused of environmental rule violations...,CNBC,2021-04-28 10:31:12.850178-04:00,The EPA accused Tesla last week of failing to ...,Lora Kolodny,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-accused-...,419.804284
1,U.K. Must Create More Projects to Attract Inve...,Bloomberg,2021-04-28 07:19:57.024376-04:00,"Source: Bloomberg, 21:47",,https://mertium.s3.amazonaws.com/media/media/b...,https://www.bloomberg.com/news/audio/2021-04-2...,400.000000
2,"Covid Stricken India Receives Ventilators, Int...",Forbes,2021-04-28 08:22:58.750101-04:00,"Several countries including the U.S., U.K., Ru...",Siladitya Ray,https://mertium.s3.amazonaws.com/media/media/0...,https://www.forbes.com/sites/siladityaray/2021...,402.000000
3,Goldman Sachs thinks the U.K. economy will gro...,MarketWatch,2021-04-28 06:16:04.702386-04:00,Early indications show the country is bouncing...,Rupert Steiner,https://mertium.s3.amazonaws.com/media/media/s...,https://www.marketwatch.com/story/goldman-sach...,421.099433
4,Tesla is now sitting on $2.5 billion of bitcoin,CNBC,2021-04-28 07:19:17.623325-04:00,The company announced earlier this year that i...,Jesse Pound,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-is-now-s...,400.800000


### Text Preprocessing

In [16]:
stop_words = set(stopwords.words('english'))

dfx['TITLE'] = ""

for i in range(len(dfx)):
    string = ""
    for word in dfx["HEADLINE"][i].split():
        word = ("".join(e for e in word if e.isalnum()))
        word = word.lower()
        if not word in stop_words:
            string += word + " "  
    dfx.at[i,"TITLE"] = string.strip()
dfx.head()

,HEADLINE,NEWS_SOURCE,PUBLISHED,DESCRIPTION,AUTHOR,IMAGE_URL,LINK,TRENDING_SCORE,TITLE
0,Tesla accused of environmental rule violations...,CNBC,2021-04-28 10:31:12.850178-04:00,The EPA accused Tesla last week of failing to ...,Lora Kolodny,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-accused-...,419.804284,tesla accused environmental rule violations us...
1,U.K. Must Create More Projects to Attract Inve...,Bloomberg,2021-04-28 07:19:57.024376-04:00,"Source: Bloomberg, 21:47",,https://mertium.s3.amazonaws.com/media/media/b...,https://www.bloomberg.com/news/audio/2021-04-2...,400.000000,uk must create projects attract investment gri...
2,"Covid Stricken India Receives Ventilators, Int...",Forbes,2021-04-28 08:22:58.750101-04:00,"Several countries including the U.S., U.K., Ru...",Siladitya Ray,https://mertium.s3.amazonaws.com/media/media/0...,https://www.forbes.com/sites/siladityaray/2021...,402.000000,covid stricken india receives ventilators inte...
3,Goldman Sachs thinks the U.K. economy will gro...,MarketWatch,2021-04-28 06:16:04.702386-04:00,Early indications show the country is bouncing...,Rupert Steiner,https://mertium.s3.amazonaws.com/media/media/s...,https://www.marketwatch.com/story/goldman-sach...,421.099433,goldman sachs thinks uk economy grow faster us...
4,Tesla is now sitting on $2.5 billion of bitcoin,CNBC,2021-04-28 07:19:17.623325-04:00,The company announced earlier this year that i...,Jesse Pound,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-is-now-s...,400.800000,tesla sitting 25 billion bitcoin


In [17]:
lemmatizer = WordNetLemmatizer()

for i in range(len(dfx)):
    string = ""
    for w in word_tokenize(dfx["TITLE"][i]):
        string += lemmatizer.lemmatize(w,pos = "v") + " "
    dfx.at[i, "TITLE"] = string.strip()
dfx.head()

,HEADLINE,NEWS_SOURCE,PUBLISHED,DESCRIPTION,AUTHOR,IMAGE_URL,LINK,TRENDING_SCORE,TITLE
0,Tesla accused of environmental rule violations...,CNBC,2021-04-28 10:31:12.850178-04:00,The EPA accused Tesla last week of failing to ...,Lora Kolodny,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-accused-...,419.804284,tesla accuse environmental rule violations us ...
1,U.K. Must Create More Projects to Attract Inve...,Bloomberg,2021-04-28 07:19:57.024376-04:00,"Source: Bloomberg, 21:47",,https://mertium.s3.amazonaws.com/media/media/b...,https://www.bloomberg.com/news/audio/2021-04-2...,400.000000,uk must create project attract investment grim...
2,"Covid Stricken India Receives Ventilators, Int...",Forbes,2021-04-28 08:22:58.750101-04:00,"Several countries including the U.S., U.K., Ru...",Siladitya Ray,https://mertium.s3.amazonaws.com/media/media/0...,https://www.forbes.com/sites/siladityaray/2021...,402.000000,covid stricken india receive ventilators inter...
3,Goldman Sachs thinks the U.K. economy will gro...,MarketWatch,2021-04-28 06:16:04.702386-04:00,Early indications show the country is bouncing...,Rupert Steiner,https://mertium.s3.amazonaws.com/media/media/s...,https://www.marketwatch.com/story/goldman-sach...,421.099433,goldman sachs think uk economy grow faster us ...
4,Tesla is now sitting on $2.5 billion of bitcoin,CNBC,2021-04-28 07:19:17.623325-04:00,The company announced earlier this year that i...,Jesse Pound,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-is-now-s...,400.800000,tesla sit 25 billion bitcoin


### Word2Vec Embeddings

In [20]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from sklearn.preprocessing import OneHotEncoder 

In [19]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
print("UNK" in model)
model.vector_size

True


300

In [21]:
w2v_headline = []
for i in dfx['TITLE']:
    w2Vec_word = np.zeros(300, dtype="float32")
    word_in_model = 0
    for word in i.split():
        if word in model:
            w2Vec_word = np.add(w2Vec_word, model[word])
            word_in_model += 1
    w2Vec_word = np.divide(w2Vec_word, word_in_model)
    w2v_headline.append(w2Vec_word)
w2v_headline = np.array(w2v_headline)

In [22]:
w2v_headline.shape

(890, 300)

In [23]:
print(dfx['NEWS_SOURCE'].unique())
news_source_onehot_encoded = OneHotEncoder().fit_transform(np.array(dfx["NEWS_SOURCE"]).reshape(-1,1))
news_source_onehot_encoded

['CNBC' 'Bloomberg' 'Forbes' 'MarketWatch' 'Financial Times' 'WSJ' 'NYT'
 'Fed in Print' 'Seeking Alpha']


<890x9 sparse matrix of type '<class 'numpy.float64'>'
	with 890 stored elements in Compressed Sparse Row format>

In [26]:
dfx[dfx.index == 0]

,HEADLINE,NEWS_SOURCE,PUBLISHED,DESCRIPTION,AUTHOR,IMAGE_URL,LINK,TRENDING_SCORE,TITLE
0,Tesla accused of environmental rule violations...,CNBC,2021-04-28 10:31:12.850178-04:00,The EPA accused Tesla last week of failing to ...,Lora Kolodny,https://mertium.s3.amazonaws.com/media/media/1...,https://www.cnbc.com/2021/04/28/tesla-accused-...,419.804284,tesla accuse environmental rule violations us ...


In [ ]:
Index2Article = {}
Article2Index = {}

clickedArticleIDs = [0, 3]
for articleID in clickedArticleIDs:
    Index2Article[dfx[dfx.index == articleID].index[0]] = articleID
    Article2Index[articleID] = dfx[dfx.index == articleID].index[0]
# clickedArticleDict
Index2Article
Article2Index

In [40]:
def avg_w2v_with_category(row_index, num_similar_items, w1,w2): #headline_preference = True, category_preference = False):
    w2v_dist  = pairwise_distances(w2v_headline, w2v_headline[row_index].reshape(1,-1))
    category_dist = pairwise_distances(news_source_onehot_encoded, news_source_onehot_encoded[row_index]) + 1
    
    w2v_cos = cosine_similarity(w2v_headline,w2v_headline[row_index].reshape(1,-1))
    category_cos = cosine_similarity(news_source_onehot_encoded,news_source_onehot_encoded[row_index])
    
    weighted_couple_dist   = (w1 * w2v_dist +  w2 * category_dist)/float(w1 + w2)
    indices_dist = np.argsort(weighted_couple_dist.flatten())[0:num_similar_items].tolist()
    
    weighted_couple_cos   = (w1 * w2v_cos +  w2 * category_cos)/float(w1 + w2)
    indices_cos = np.argsort(weighted_couple_cos.ravel())[::-1][0:num_similar_items]
    
    indices = indices_cos
#     indices = indices_dist
    dfr = pd.DataFrame({
                'Id': dfx.index[indices].values, 
                'Title':dfx['TITLE'][indices].values,
                'Weighted Euclidean Distance': weighted_couple_dist[indices].ravel(),
                'Weighted Cosine Similarity': weighted_couple_cos[indices].ravel(),
                'News Source': dfx['NEWS_SOURCE'][indices].values,
                'Published Date': dfx['PUBLISHED'][indices].values,
                }).set_index('Id', drop=True)
    
    print("="*23,"Queried article details","="*23)
    print('Title : ',dfx['HEADLINE'][indices[0]])
    print('News Source : ', dfx['NEWS_SOURCE'][indices[0]])
    print("\n","="*23,"Recommended articles : ","="*23)
    display(dfr.iloc[1:, ])
    return dfr.iloc[1:, ]



In [44]:
dfr = avg_w2v_with_category(3,10,1,0)

======================= Queried article details =======================
Title :  Goldman Sachs thinks the U.K. economy will grow faster than the U.S. — here’s why
News Source :  MarketWatch

 ======================= Recommended articles :  =======================


,Title,Weighted Euclidean Distance,Weighted Cosine Similarity,News Source,Published Date
Id,,,,,
500,new economy saturday maybe china wont beat us,0.959558,0.722889,Bloomberg,2021-05-01 12:05:49.918797
253,biden want reform americas break unemployment ...,0.973856,0.714763,MarketWatch,2021-04-30 19:36:42.479861
812,goldman bring us uk bankers back office june,0.982480,0.704693,Financial Times,2021-05-04 17:25:35.419971
430,jamie dimon boom economy finally get zoom,1.130124,0.686374,WSJ,2021-05-04 15:22:48.422314
749,nfls richard sherman see exponential growth es...,1.085252,0.684658,Bloomberg,2021-05-03 19:40:31.163956
89,economic report us economy speed first quarter...,1.022934,0.680571,MarketWatch,2021-04-28 17:44:09.466413
370,barclays ceo see strongest uk growth since lea...,1.063970,0.674469,Bloomberg,2021-04-30 14:20:29.918038
537,americans cant get enough stock market,1.165904,0.670390,WSJ,2021-05-02 09:38:08.632672
696,small businesses slowly recover financial help...,1.011195,0.666403,CNBC,2021-05-03 13:24:57.977650


In [42]:
list(dfr['Title'])

['new economy saturday maybe china wont beat us',
 'biden want reform americas break unemployment system heres may look',
 'goldman bring us uk bankers back office june',
 'jamie dimon boom economy finally get zoom',
 'nfls richard sherman see exponential growth esports',
 'economic report us economy speed first quarter heres gdp data show',
 'barclays ceo see strongest uk growth since least 1948',
 'americans cant get enough stock market',
 'small businesses slowly recover financial help become target heres whats available']

In [36]:
dfr = avg_w2v_with_category(2,10,1,0)

======================= Queried article details =======================
Title :  Covid Stricken India Receives Ventilators, International Aid As Several Countries Promise Help
News Source :  Forbes

 ======================= Recommended articles :  =======================


,Title,Weighted Euclidean Distance,Weighted Cosine Similarity,News Source,Published Date
Id,,,,,
260,biden restrict travel us covid stricken india ...,1.084944,0.582765,Forbes,2021-04-30 20:39:17.438000
157,us promise send crucial oxygen supply covid ra...,1.093359,0.641575,Forbes,2021-04-29 02:11:20.552875
803,aid pour india around world covid case soar,1.108002,0.619718,Financial Times,2021-05-04 13:16:07.194698
324,personal finance daily letter india truck carr...,1.116421,0.510855,MarketWatch,2021-04-29 21:18:13.855250
693,market extra rural america mutual fund dedicat...,1.138939,0.519982,MarketWatch,2021-05-03 13:25:20.313079
681,private effort aid launch feed digital dollar ...,1.141346,0.550111,WSJ,2021-05-03 12:23:46.945617
696,small businesses slowly recover financial help...,1.145971,0.529198,CNBC,2021-05-03 13:24:57.977650
742,clorox still sell many clean products pre pand...,1.177767,0.497741,MarketWatch,2021-05-03 18:36:11.807238
605,impact invest might help save world get messy ...,1.186818,0.482844,Bloomberg,2021-05-04 12:14:20.662173


In [37]:
list(dfr['Title'])

['biden restrict travel us covid stricken india begin may 4',
 'us promise send crucial oxygen supply covid ravage india',
 'aid pour india around world covid case soar',
 'personal finance daily letter india truck carry oxygen cylinders waylay loot pending home sales rise low inventory could cause headaches buyers',
 'market extra rural america mutual fund dedicate long await revival',
 'private effort aid launch feed digital dollar accelerate work',
 'small businesses slowly recover financial help become target heres whats available',
 'clorox still sell many clean products pre pandemic',
 'impact invest might help save world get messy along way']